In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import time

#        buscar_animal_input = self.browser.find_element_by_css_selector('input#buscar-animal')

In [2]:
#driver_path = 'C:/Program Files (x86)/Microsoft/Edge/Application/msedgedriver.exe'
driver = webdriver.Edge()

In [3]:
# Using a list comprehension
start_value = 49
step = 48
n = 42  # Number of times to add the step

range_index = [start_value + i * step for i in range(n)]


In [4]:
lista_paginas = [(f'https://lista.mercadolivre.com.br/calcados-roupas-bolsas/roupas_Desde_{index}_NoIndex_True') for index in range_index]

In [5]:
URL_catalogo = 'https://lista.mercadolivre.com.br/roupas#D[A:roupas]'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.76",
           "Accept-Language": "pt-BR,pt;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,ja;q=0.5,es;q=0.4"}

In [6]:
page = requests.get(URL_catalogo)
soup = BeautifulSoup(page.content, 'html.parser')
soup = BeautifulSoup(soup.prettify(), 'html.parser')

In [7]:
links_roupas = []

for url in lista_paginas:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    for item in soup.find_all(class_='ui-search-item__group__element shops__items-group-details ui-search-link'):
        links_roupas.append(item.get('href'))
    
    print(f'Número de links: {len(links_roupas)}')

Número de links: 54
Número de links: 108
Número de links: 162
Número de links: 216
Número de links: 270
Número de links: 324
Número de links: 378
Número de links: 432
Número de links: 486
Número de links: 540
Número de links: 594
Número de links: 648
Número de links: 702
Número de links: 756
Número de links: 810
Número de links: 864
Número de links: 918
Número de links: 972
Número de links: 1026
Número de links: 1080
Número de links: 1134
Número de links: 1188
Número de links: 1242
Número de links: 1296
Número de links: 1350
Número de links: 1404
Número de links: 1458
Número de links: 1512
Número de links: 1566
Número de links: 1620
Número de links: 1668
Número de links: 1722
Número de links: 1776
Número de links: 1830
Número de links: 1884
Número de links: 1938
Número de links: 1992
Número de links: 2046
Número de links: 2100
Número de links: 2154
Número de links: 2208
Número de links: 2208


In [8]:
colunas = ['Título', 'Reais', 'Centavos', 'Condição', 'Nota', 'N_Avaliações', 'Desconto', 'Marca', 'Material', 'Gênero', 'Temporada', 'Review1', 'Review2', 'Review3']

#with open('webscrap_ml.csv' )

In [9]:
lista_de_textos = ['Marca:', 'Material principal:', 'Gênero:', 'Temporada de lançamento:']
elementos_basicos = [{'name':'h1', 'class':'ui-pdp-title'}, {'name':'span', 'class':'andes-money-amount__fraction'}, {'name':'span', 'class':'andes-money-amount__cents andes-money-amount__cents--superscript-36'},
                     {'name':'span', 'class':'ui-pdp-subtitle'}, {'name':'span', 'class':'ui-pdp-review__rating'}, {'name':'span', 'class':'ui-pdp-review__amount'},
                     {'name':'span', 'class':'andes-money-amount__discount'}]



def insere_dados_basicos(elementos, soup):
    lista = []
    for item in elementos:
        try:
            elemento = soup.find(item['name'], class_=item['class']).get_text().strip()
            lista.append(elemento)
        except:
            lista.append(None)
        
    return lista

def insere_elementos_tabela(lista_de_textos, soup):
    lista = []

    for elemento in lista_de_textos:
        elemento = soup.find('span', text=elemento)
        
        if elemento:
            elemento = elemento.find_next_sibling('span').text.strip()
        else:
            elemento = "N/A"

        lista.append(elemento)
    return lista

def insere_reviews(elemento_nome, elemento_classe, soup):
    lista_reviews = []
    reviews = soup.find_all(name=elemento_nome, class_=elemento_classe)
    for review in reviews:
        lista_reviews.append(review.get_text().strip())
        
    return lista_reviews

In [19]:
'''page = driver.get('https://produto.mercadolivre.com.br/MLB-3168301568-short-jeans-feminino-cintura-alta-com-lycra-barra-desfiada-_JM#position=4&search_layout=grid&type=item&tracking_id=4c978608-655d-4a2f-bb90-dcca8a499d3a')
cod_page = driver.page_source
soup = BeautifulSoup(cod_page, 'html.parser')
#comentarios = soup.find_all(name='p', class_='ui-review-capability-comments__comment__content')[1].get_text()
#print(comentarios)'''

In [10]:
lista = []

for link in links_roupas:
    page = driver.get(link)
    cod_page = driver.page_source
    soup = BeautifulSoup(cod_page, 'html.parser')
    lista_basicos = insere_dados_basicos(elementos_basicos, soup)
    lista_caracteristica = insere_elementos_tabela(lista_de_textos, soup)
    lista_reviews = insere_reviews('p', 'ui-review-capability-comments__comment__content', soup)
    lista.append(lista_basicos + lista_caracteristica + lista_reviews)
    print(f'n obs: {len(lista)}')
    time.sleep(15)

C:\Users\Pichau\AppData\Local\Temp\ipykernel_19944\1821577957.py:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elemento = soup.find('span', text=elemento)


n obs: 1
n obs: 2
n obs: 3
n obs: 4
n obs: 5
n obs: 6
n obs: 7
n obs: 8
n obs: 9
n obs: 10
n obs: 11
n obs: 12
n obs: 13
n obs: 14
n obs: 15
n obs: 16
n obs: 17
n obs: 18
n obs: 19
n obs: 20
n obs: 21
n obs: 22
n obs: 23
n obs: 24
n obs: 25
n obs: 26
n obs: 27
n obs: 28
n obs: 29
n obs: 30
n obs: 31
n obs: 32
n obs: 33
n obs: 34
n obs: 35
n obs: 36
n obs: 37
n obs: 38
n obs: 39
n obs: 40
n obs: 41
n obs: 42
n obs: 43
n obs: 44
n obs: 45
n obs: 46
n obs: 47
n obs: 48
n obs: 49
n obs: 50
n obs: 51
n obs: 52
n obs: 53
n obs: 54
n obs: 55
n obs: 56
n obs: 57
n obs: 58
n obs: 59
n obs: 60
n obs: 61
n obs: 62
n obs: 63
n obs: 64
n obs: 65
n obs: 66
n obs: 67
n obs: 68
n obs: 69
n obs: 70
n obs: 71
n obs: 72
n obs: 73
n obs: 74
n obs: 75
n obs: 76
n obs: 77
n obs: 78
n obs: 79
n obs: 80
n obs: 81
n obs: 82
n obs: 83
n obs: 84
n obs: 85
n obs: 86
n obs: 87
n obs: 88
n obs: 89
n obs: 90
n obs: 91
n obs: 92
n obs: 93
n obs: 94
n obs: 95
n obs: 96
n obs: 97
n obs: 98
n obs: 99
n obs: 100
n obs: 1

KeyboardInterrupt: 

In [11]:
for link in links_roupas[1629:]:
    page = driver.get(link)
    cod_page = driver.page_source
    soup = BeautifulSoup(cod_page, 'html.parser')
    lista_basicos = insere_dados_basicos(elementos_basicos, soup)
    lista_caracteristica = insere_elementos_tabela(lista_de_textos, soup)
    lista_reviews = insere_reviews('p', 'ui-review-capability-comments__comment__content', soup)
    lista.append(lista_basicos + lista_caracteristica + lista_reviews)
    print(f'n obs: {len(lista)}')
    time.sleep(9)

C:\Users\Pichau\AppData\Local\Temp\ipykernel_19944\1821577957.py:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  elemento = soup.find('span', text=elemento)


n obs: 872
n obs: 873
n obs: 874
n obs: 875
n obs: 876
n obs: 877
n obs: 878
n obs: 879
n obs: 880
n obs: 881
n obs: 882
n obs: 883
n obs: 884
n obs: 885
n obs: 886
n obs: 887
n obs: 888
n obs: 889
n obs: 890
n obs: 891
n obs: 892
n obs: 893
n obs: 894
n obs: 895
n obs: 896
n obs: 897
n obs: 898
n obs: 899
n obs: 900
n obs: 901
n obs: 902
n obs: 903
n obs: 904
n obs: 905
n obs: 906
n obs: 907
n obs: 908
n obs: 909
n obs: 910
n obs: 911
n obs: 912
n obs: 913
n obs: 914
n obs: 915
n obs: 916
n obs: 917
n obs: 918
n obs: 919
n obs: 920
n obs: 921
n obs: 922
n obs: 923
n obs: 924
n obs: 925
n obs: 926
n obs: 927
n obs: 928
n obs: 929
n obs: 930
n obs: 931
n obs: 932
n obs: 933
n obs: 934
n obs: 935
n obs: 936
n obs: 937
n obs: 938
n obs: 939
n obs: 940
n obs: 941
n obs: 942
n obs: 943
n obs: 944
n obs: 945
n obs: 946
n obs: 947
n obs: 948
n obs: 949
n obs: 950
n obs: 951
n obs: 952
n obs: 953
n obs: 954
n obs: 955
n obs: 956
n obs: 957
n obs: 958
n obs: 959
n obs: 960
n obs: 961
n obs: 962

KeyboardInterrupt: 

In [12]:
df = pd.DataFrame(lista, columns=colunas)

In [13]:
df

,Título,Reais,Centavos,Condição,Nota,N_Avaliações,Desconto,Marca,Material,Gênero,Temporada,Review1,Review2,Review3
0,Pijama Cirurgico Scrub Roupa Privativa Hospit...,69,90,Novo | +1000 vendidos,4.0,(269),None,TORONTO UNIFORMES PROFISSIONAIS,Poliéster,Sem gênero,N/A,"Preço justo , bem feito acabamento ok. Única s...","Tamanho bom, costuras de qualidade, porém teci...","Muito bom, só poderia ter os cortes na lateral..."
1,Kit 6 Cuecas Masculino Slip Mash De Algodão Al...,99,99,Novo | +1000 vendidos,4.7,(292),27% OFF,Mash,Algodão,Masculino,Primavera-verão Outono-inverno,"Produto de qualidade, recomendo a todos do mer...",Eu recomendo.,Bom produto.
2,Kit 10 Cuecas Boxer Lupo Cueca Box Algodão Mas...,232,89,Novo | +10mil vendidos,4.5,(3026),18% OFF,Lupo,Algodão,Masculino,Outono/Inverno,As cuecas são boas; porém você percebe na cost...,"Pelo preço promocional, com ctz tem algum defe...","Serviram muito bem, porém é nítido que o pano ..."
3,Cinto Porta Número E Gel Para Corrida - Ciacool,36,90,Novo | +500 vendidos,4.8,(37),None,CiaCool,N/A,Sem gênero,N/A,"Ótimo, perfeito para o que eu precisava.",Material de boa qualidade. Simples e pratico.,Excelente.
4,Calça Preta Ou Azul Uniforme Tactel 3 Bolsos,26,90,Novo | +1000 vendidos,4.0,(18),None,C. SURF,432261,Masculino,Primavera/Verão,"Gostei muito. Compensa demais. Tecido leve, be...","Produto de baixa qualidade, tecido muito fraco...","Perfeita , pelo preço é ótima, comprei pra usa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,Short Jeans Easy Lança Perfume Comfort Low Iv...,530,74,Novo | 3 vendidos,None,None,None,Easy Lança Perfume,Jeans,Feminino,Outono/Inverno,None,None,None
1624,Roupa Infantil Menina Estilosa Blogueirinha,78,16,Novo | +500 vendidos,4.9,(145),6% OFF,PROPRIA,N/A,Meninas,N/A,Amei eu vou comprar mas.,"Produto exelente, tecido ótimo, de exelente qu...","Lindo demais , minha filha amou."
1625,Kit Conjunto Infantil Do Cristiano Ronaldo,79,None,Novo | +5 vendidos,None,None,None,Tg,N/A,Masculino,N/A,None,None,None
1626,Roupas Infantis Menina Brinquedo Poppett Toy F...,89,10,Novo | +50 vendidos,4.0,(10),None,Thalyane Teen Kids,N/A,Meninas,N/A,O conjunto é bonito mais o tamanho do short é ...,Produto como na foto mt lindo.\nMinha filha am...,"Tecido muito ruim, não recomendo e não comprar..."


In [14]:
df.shape

(1628, 14)

In [19]:
df.to_csv('scrapping_ate_1628.csv', encoding='utf-8')

In [20]:
df2 = pd.read_csv('scrapping_ate_1628.csv')

In [21]:
df2

,Unnamed: 0,Título,Reais,Centavos,Condição,Nota,N_Avaliações,Desconto,Marca,Material,Gênero,Temporada,Review1,Review2,Review3
0,0,Pijama Cirurgico Scrub Roupa Privativa Hospit...,69.0,90.0,Novo | +1000 vendidos,4.0,(269),NaN,TORONTO UNIFORMES PROFISSIONAIS,Poliéster,Sem gênero,NaN,"Preço justo , bem feito acabamento ok. Única s...","Tamanho bom, costuras de qualidade, porém teci...","Muito bom, só poderia ter os cortes na lateral..."
1,1,Kit 6 Cuecas Masculino Slip Mash De Algodão Al...,99.0,99.0,Novo | +1000 vendidos,4.7,(292),27% OFF,Mash,Algodão,Masculino,Primavera-verão Outono-inverno,"Produto de qualidade, recomendo a todos do mer...",Eu recomendo.,Bom produto.
2,2,Kit 10 Cuecas Boxer Lupo Cueca Box Algodão Mas...,232.0,89.0,Novo | +10mil vendidos,4.5,(3026),18% OFF,Lupo,Algodão,Masculino,Outono/Inverno,As cuecas são boas; porém você percebe na cost...,"Pelo preço promocional, com ctz tem algum defe...","Serviram muito bem, porém é nítido que o pano ..."
3,3,Cinto Porta Número E Gel Para Corrida - Ciacool,36.0,90.0,Novo | +500 vendidos,4.8,(37),NaN,CiaCool,NaN,Sem gênero,NaN,"Ótimo, perfeito para o que eu precisava.",Material de boa qualidade. Simples e pratico.,Excelente.
4,4,Calça Preta Ou Azul Uniforme Tactel 3 Bolsos,26.0,90.0,Novo | +1000 vendidos,4.0,(18),NaN,C. SURF,432261,Masculino,Primavera/Verão,"Gostei muito. Compensa demais. Tecido leve, be...","Produto de baixa qualidade, tecido muito fraco...","Perfeita , pelo preço é ótima, comprei pra usa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,1623,Short Jeans Easy Lança Perfume Comfort Low Iv...,530.0,74.0,Novo | 3 vendidos,NaN,NaN,NaN,Easy Lança Perfume,Jeans,Feminino,Outono/Inverno,NaN,NaN,NaN
1624,1624,Roupa Infantil Menina Estilosa Blogueirinha,78.0,16.0,Novo | +500 vendidos,4.9,(145),6% OFF,PROPRIA,NaN,Meninas,NaN,Amei eu vou comprar mas.,"Produto exelente, tecido ótimo, de exelente qu...","Lindo demais , minha filha amou."
1625,1625,Kit Conjunto Infantil Do Cristiano Ronaldo,79.0,NaN,Novo | +5 vendidos,NaN,NaN,NaN,Tg,NaN,Masculino,NaN,NaN,NaN,NaN
1626,1626,Roupas Infantis Menina Brinquedo Poppett Toy F...,89.0,10.0,Novo | +50 vendidos,4.0,(10),NaN,Thalyane Teen Kids,NaN,Meninas,NaN,O conjunto é bonito mais o tamanho do short é ...,Produto como na foto mt lindo.\nMinha filha am...,"Tecido muito ruim, não recomendo e não comprar..."


In [37]:
lista[3][0]

'N'